В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'default_ON_QnSC_200k'
dataname_initial = 'default'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'

idx = list(range(10))
constants = [0, 0.001, 0.01, 0.1, 0.25, 0.5]

CONFIG.add_arg('save_cat', None)
target_cat=True

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
for sigma in constants:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/default_ON_QnSC_200k` folder


In [7]:
CONFIG.get_all_args()

{'dataname': 'default_ON_QnSC_200k',
 'method': 'tabddpm',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/default/tabddpm.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 66,
 'real_path': './synthetic/default_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/default_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/default_ON_QnSC_200k/initial_info.json',
 'save_path': './synthetic/shoppers_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_mult_9.csv',
 'num_clusters': 25,
 'save_cat': None}

### Sample + Eval

In [10]:
sample_save_path

'./synthetic/default_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv'

In [ ]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers, target_cat=target_cat)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False)
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/default/tabddpm_ON_QnSC_200k_const_0 tabddpm_ON_QnSC_200k_const_0 ./synthetic/default_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 24/24 [00:00<00:00, 54.90it/s]|
Column Shapes Score: 98.71%

(2/2) Evaluating Column Pair Trends: |██████████| 276/276 [00:02<00:00, 97.22it/s]| 
Column Pair Trends Score: 97.97%

Overall Score (Average): 98.34%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0: 1.294 ± 0.525
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0: 2.029 ± 1.683


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!
./models/tabddpm_ON_QnSC/ckpt/default/tabddpm_ON_QnSC_200k_const_0_001 tabddpm_ON_QnSC_200k_const_0_001 ./synthetic/default_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_001.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 24/24 [00:00<00:00, 59.17it/s]|
Column Shapes Score: 98.87%

(2/2) Evaluating Column Pair Trends: |██████████| 276/276 [00:02<00:00, 92.08it/s]| 
Column Pair Trends Score: 98.16%

Overall Score (Average): 98.52%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0_001: 1.128 ± 0.454
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0_001: 1.842 ± 1.840


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


In [9]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname_initial}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [11]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                              Type     Data   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  default   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  default   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  default   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  default   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  default   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  default   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  default   

       base_metrics                                                  \
  Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0            0.4855             0.4747        0.4917         0.4726   
0            0.4855             0.4798        0.4917         0.4436   
0            0.4855             0.4781        0.4920         0.4646   
0            0.4855             0.4843        0.4917         0.4638   
0            0.4855             0.4855        0.4917         0.4833   
0            0.4855             0.4809        0.4917         0.4494   
0            0.4855             0.4797        0.4917         0.4703   
0            0.4855             0.4798        0.4917         0.4464   
0            0.4855             0.4825        0.4917         0.4450   
0            0.4855             0.4831        0.4917         0.4339   
0            0.4855             0.4699        0.4917         0.4477   
0            0.4855             0.4657        0.4917         0.4531   
0            0.4855             0.4841        0.4917         0.4962   
0            0.4855             0.4801        0.4917         0.4261   
0            0.4855             0.4788        0.4917         0.4714   
0            0.4855             0.4811        0.4917         0.4990   

                                                              similarity  \
  Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                    2.2344                3.8821                99.0273   
0                    1.1761                9.7745                98.8724   
0                    1.5225                5.5503                98.7065   
0                    0.2463                5.6691                98.4269   
0                    0.0117                1.7037                98.4727   
0                    0.9462                8.6021                98.6201   
0                    1.2026                4.3484                98.5256   
0                    1.1756                9.2164                98.4867   
0                    0.6193                9.4929                98.4256   
0                    0.5109               11.7522                98.5708   
0                    3.2135                8.9480                98.6211   
0                    4.0775                7.8488                98.3603   
0                    0.2902               -0.9242                98.5688   
0                    1.1220               13.3413                98.4762   
0                    1.3855                4.1255                98.2324   
0                    0.9091               -1.4836                97.1633   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                     98.3867                    98.7070   
0                     98.1579                    98.5151   
0   

In [12]:
import os
if not os.path.exists(f'./eval/total/{dataname_initial}'):
    os.makedirs(f'./eval/total/{dataname_initial}')
final_metrics_table.to_csv(f'./eval/total/{dataname_initial}/{model_short}{model_name_postfix}_final_metrics_table.csv', index=False)